In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [3]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.6, 1.6, 1.7), (1.75, 1.75), (1.55, 1.55, 1.6, 1.85)]
  wgt = [(0.7, 0.3, 0.7), (0.7, 0.6), (0.4, 0.5, 0.8, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.188044                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.112765
0.000750      2.0  0.214588                  2.0  0.137949
0.001052      3.0  0.293138                  3.0  0.196982
0.001476      4.0  0.379804                  4.0  0.263217
0.002070      5.0  0.472872                  5.0  0.335907
0.002904      6.0  0.570643                  6.0  0.414744
0.004074      7.0  0.670541                  7.0  0.502073
0.005714      8.0  0.769042                  8.0  0.596408
0.008015      9.0  0.861818                  9.0  0.693860
0.011243     10.0  0.943534                 10.0  0.790947
0.015771     11.0  1.008420                 11.0  0.882907
0.022122     12.0  1.051216                 12.0  0.963257
0.031031     13.0  1.068546                 13.0  1.022644
0.043528     14.0  1.036763                 14.0  1.032112
0.061057     15.0  0.942239                 15.0  0.967895
0.085645     16.0  0.823091                 16.0  0.850195
0.120136     17.0  0.713015                 17.0  0.709616
0.168516     18.0  0.622430                 18.0  0.573645
0.236378     19.0  0.558470                 19.0  0.474565
0.331549     20.0  0.527826                 20.0  0.435737
0.465100     21.0  0.544308                 21.0  0.467381
0.652400     22.0  0.491061                 22.0  0.451616
0.915100     23.0  0.359247                 23.0  0.357004
1.283650     24.0  0.264390                 24.0  0.276037
1.800600     25.0  0.206347                 25.0  0.217468
2.525700     26.0  0.163279                 26.0  0.171613
3.542800     27.0  0.126308                 27.0  0.137227
4.969550     28.0  0.100435                 28.0  0.116573
6.970850     29.0  0.087924                 29.0  0.109563
9.778100     30.0  0.078098                 30.0  0.100825
13.715850    31.0  0.066220                 31.0  0.084394
19.239350    32.0  0.053203                 32.0  0.063031
26.987250    33.0  0.047291                 33.0  0.049551
37.855300    34.0  0.048736                 34.0  0.046736
53.100050    35.0  0.049165                 35.0  0.049245
73.887500    36.0  0.048824                 36.0  0.055379
97.662500    37.0  0.048020                 37.0  0.058250
121.437500   38.0  0.044461                 38.0  0.054538
145.212500   39.0  0.039541                 39.0  0.047630
168.987500   40.0  0.035310                 40.0  0.041151
192.762500   41.0  0.031101                 41.0  0.034971
216.537500   42.0  0.026651                 42.0  0.028992
240.312500   43.0  0.021669                 43.0  0.022582
264.087500   44.0  0.015451                 44.0  0.014953
287.862500   45.0  0.007170                 45.0  0.005417
311.637500   46.0  0.004500                 46.0  0.002073
335.412500   47.0  0.012056                 47.0  0.009752
359.187500   48.0  0.021783                 48.0  0.019856
382.962500   49.0  0.029550                 49.0  0.027875
406.737500   50.0  0.036388                 50.0  0.034690
430.512500   51.0  0.042684                 51.0  0.040832
454.287500   52.0  0.048586                 52.0  0.046617
478.062500   53.0  0.054169                 53.0  0.052203
501.837500   54.0  0.059460                 54.0  0.057603
525.612500   55.0  0.064458                 55.0  0.062747
549.387500   56.0  0.069116                 56.0  0.067779
573.162500   57.0  0.073369                 57.0  0.072576
596.937500   58.0  0.077126                 58.0  0.076995
620.712500   59.0  0.080298                 59.0  0.080878
644.487500   60.0  0.082830                 60.0  0.084114
668.262500   61.0  0.084719                 61.0  0.086667
692.037500   62.0  0.086063                 62.0  0.088672
715.812500   63.0  0.087001                 63.0  0.090183
739.587500   64.0  0.087736                 64.0  0.091372
76

# Fluxes by Layer

CRD                          CLIRAD             \
                         flug      fldg      fnetg       flug       fldg   
pressure     level                                                         
0.000000e+00 1     -20.835452  0.000000 -20.835452        NaN        NaN   
1.000000e-08 1            NaN       NaN        NaN -20.853089   0.000002   
6.244000e-04 2     -20.835486  0.000048 -20.835438 -20.853117   0.000038   
8.759000e-04 3     -20.835500  0.000069 -20.835432 -20.853128   0.000053   
1.228600e-03 4     -20.835519  0.000100 -20.835419 -20.853144   0.000077   
1.723400e-03 5     -20.835545  0.000148 -20.835397 -20.853165   0.000113   
2.417400e-03 6     -20.835580  0.000222 -20.835358 -20.853193   0.000169   
3.390900e-03 7     -20.835627  0.000334 -20.835292 -20.853230   0.000255   
4.756500e-03 8     -20.835688  0.000504 -20.835184 -20.853280   0.000386   
6.672000e-03 9     -20.835768  0.000758 -20.835009 -20.853345   0.000586   
9.358900e-03 10    -20.835869  0.001134 -20.834735 -20.853428   0.000890   
1.312780e-02 11    -20.835996  0.001682 -20.834314 -20.853531   0.001346   
1.841450e-02 12    -20.836149  0.002467 -20.833682 -20.853654   0.002022   
2.583020e-02 13    -20.836325  0.003566 -20.832759 -20.853796   0.003010   
3.623230e-02 14    -20.836512  0.005070 -20.831442 -20.853943   0.004418   
5.082340e-02 15    -20.836695  0.007045 -20.829650 -20.854085   0.006343   
7.129060e-02 16    -20.836873  0.009507 -20.827366 -20.854215   0.008821   
1.000000e-01 17    -20.837038  0.012471 -20.824567 -20.854324   0.011821   
1.402710e-01 18    -20.837152  0.015986 -20.821165 -20.854374   0.015257   
1.967600e-01 19    -20.837144  0.020144 -20.817000 -20.854300   0.019021   
2.759970e-01 20    -20.836876  0.025118 -20.811758 -20.853977   0.023154   
3.871000e-01 21    -20.836036  0.031224 -20.804812 -20.853134   0.028045   
5.431000e-01 22    -20.833675  0.038922 -20.794753 -20.850922   0.034472   
7.617000e-01 23    -20.829442  0.047405 -20.782037 -20.846965   0.042210   
1.068500e+00 24    -20.824449  0.055467 -20.768981 -20.842195   0.050416   
1.498800e+00 25    -20.819235  0.063730 -20.755505 -20.837085   0.059377   
2.102400e+00 26    -20.813833  0.073082 -20.740751 -20.831653   0.069496   
2.949000e+00 27    -20.808389  0.084013 -20.724376 -20.826038   0.081094   
4.136600e+00 28    -20.803497  0.096890 -20.706607 -20.820727   0.095089   
5.802500e+00 29    -20.799655  0.112868 -20.686788 -20.816115   0.113483   
8.139200e+00 30    -20.796551  0.134100 -20.662450 -20.811842   0.139541   
1.141700e+01 31    -20.794203  0.162076 -20.632127 -20.807947   0.174799   
1.601470e+01 32    -20.793836  0.197774 -20.596062 -20.805793   0.218612   
2.246400e+01 33    -20.798642  0.243226 -20.555416 -20.808926   0.269903   
3.151050e+01 34    -20.810782  0.306044 -20.504738 -20.819712   0.333795   
4.420010e+01 35    -20.829451  0.397971 -20.431479 -20.836741   0.421084   
6.200000e+01 36    -20.855990  0.528176 -20.327814 -20.861344   0.549531   
8.577500e+01 37    -20.891006  0.700694 -20.190311 -20.894222   0.738390   
1.095500e+02 38    -20.925450  0.870377 -20.055072 -20.926721   0.934957   
1.333250e+02 39    -20.961979  1.032122 -19.929857 -20.961448   1.123297   
1.571000e+02 40    -21.003750  1.185253 -19.818497 -21.001484   1.297489   
1.808750e+02 41    -21.051398  1.332346 -19.719052 -21.047470   1.459380   
2.046500e+02 42    -21.105671  1.474211 -19.631460 -21.100339   1.610749   
2.284250e+02 43    -21.167672  1.611270 -19.556401 -21.161940   1.754009   
2.522000e+02 44    -21.239189  1.743814 -19.495376 -21.234577   1.890250   
2.759750e+02 45    -21.324009  1.872147 -19.451862 -21.322524   2.020312   
2.997500e+02 46    -21.430450  1.998780 -19.431670 -21.434145   2.147190   
3.235250e+02 47    -21.558621  2.139623 -19.418998 -21.568406   2.287288   
3.473000e+02 48    -21.696831  2.311785 -19.385046 -21.711964   2.458312   
3.710750e+02 49    -21.837747  2.514048 -19.323699 -21.857151   2.659426  

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')